# 3D interpolation

This example shows how to interpolate values from  arbitrary points in a 3D space of a function defined on a Cartesian grid.

Methods used performs an interpolation in 2D space by considering the axes of longitude and latitude of the grid, then performs a linear interpolation in the third dimension using the two values obtained by the 2D interpolation.

Let's start by building our interpolator:

In [ ]:
import xarray as xr
import pyinterp.backends.xarray as pxr

ds = xr.load_dataset("../tests/dataset/tcw.nc")
# The grid used organizes the latitudes in descending order. We ask our
# constructor to flip this axis in order to correctly evaluate the bicubic
# interpolation from this 3D cube (only necessary to perform a bicubic
# interpolation).
interpolator = pxr.Grid3D(ds.data_vars["tcw"], increasing_axes=True)
interpolator

We will build a new grid that will be used to build a new interpolated grid.

In [ ]:
import datetime
import numpy as np

# The coordinates used for interpolation are shifted to avoid using the
# points of the trivariate function.
mx, my, mz = np.meshgrid(
    np.arange(-180, 180, 0.25) + 1 / 3.0,
    np.arange(-80, 80, 0.25) + 1 / 3.0,
    np.array([datetime.datetime(2002, 7, 2, 15, 0)], dtype="datetime64"),
    indexing='ij')

We interpolate our grid using a classical [trivariate](https://pangeo-pyinterp.readthedocs.io/en/latest/generated/pyinterp.trivariate.html#pyinterp.trivariate) interpolation, then a [bicubic](https://pangeo-pyinterp.readthedocs.io/en/latest/generated/pyinterp.bicubic.html) interpolation in space followed by a linear interpolation in the temporal axis.

In [ ]:
trivariate = interpolator.trivariate(
    dict(longitude=mx.flatten(), latitude=my.flatten(), time=mz.flatten()))
bicubic = interpolator.bicubic(
    dict(longitude=mx.flatten(), latitude=my.flatten(), time=mz.flatten()))

We transform our result cubes into a matrix.

In [ ]:
trivariate = trivariate.reshape(mx.shape).squeeze(axis=2)
bicubic = bicubic.reshape(mx.shape).squeeze(axis=2)
lons = mx[:, 0].squeeze()
lats = my[0, :].squeeze()

Let's visualize our results.

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
%matplotlib inline

fig = plt.figure(figsize=(18, 9))
ax = fig.add_subplot(121, projection=ccrs.PlateCarree(central_longitude=180))
ax.pcolormesh(lons, lats, trivariate.T, cmap='jet',
              transform=ccrs.PlateCarree())
ax.coastlines()
ax.set_extent([80, 170, -45, 30], crs=ccrs.PlateCarree())
ax.set_title("Trilinear")

ax = fig.add_subplot(122, projection=ccrs.PlateCarree(central_longitude=180))
ax.pcolormesh(lons, lats, bicubic.T, cmap='jet',
              transform=ccrs.PlateCarree())
ax.coastlines()
ax.set_extent([80, 170, -45, 30], crs=ccrs.PlateCarree())
ax.set_title("Bicubic & Linear in time")